In [2]:
import re, json, pathlib, logging, time, argparse, pprint , pickle
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import postprocess.postprocess as pspr


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [ ]:
test = ['1.0','1.1','1.2','1.3','1.4','1.5','1.a','1.b']
print(test)
print(sorted(test))

In [45]:
goals = pd.read_excel('keywords/keywords_expanded.xlsx', sheet_name='Goal_keys')
targets = pd.read_excel('keywords/keywords_expanded.xlsx', sheet_name='Target_keys')
goals['Target'] = goals['Goal'].str.replace('SDG ', '')
goals['Target'] = goals['Target'].apply(lambda x: f'{x}.0')
targets['Goal'] = targets['Target'].apply(lambda x: f"SDG {x.split('.')[0]}")
keywords = pd.concat([goals, targets])
columns=list(keywords.columns.values)
for x in ['Unnamed: 0', 'Goal', 'Target']:
    columns.remove(x)
columns=['Goal','Target'] + columns
keywords = keywords.reindex(columns=columns)
keywords.sort_values(by=['Goal','Target'], inplace = True)
keywords.reset_index(inplace=True,drop=True)
keywords.to_excel('keywords/keywords_expanded_reordered.xlsx')
keywords.head(200)


,Goal,Target,0,1,2,3,4,5,6,7,...,47,48,49,50,51,52,53,54,55,56
0,SDG 1,1.0,Sustainable Development Goal 1,SDG 1,end poverty,reduce poverty,fight against poverty,end global poverty,poverty reduction,poverty and deprivation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SDG 1,1.1,international poverty line,extreme poverty,abject poverty,absolute poverty,destitution,penury,severe deprivation,help people suffering from food and material d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDG 1,1.2,severely materially deprived people,low work intensity,national poverty line,social exclusion,income poverty,combatting child poverty,children out of poverty,addressing child poverty,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDG 1,1.3,social risk,social protection,rights of persons with disabilities,rights of unemployed persons,rights of older persons,rights of pregnant women,rights of work-injury victims,rights of the poor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDG 1,1.4,tenure rights,equal rights to economic resources,control over land,access to inheritance,rights to land,\naccess to property,access to natural resources,access to new technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,SDG 9,9.4,upgrade infrastructure,sustainable industrial process,retrofitting,retrofit,resource-use efficiency,upgrade existing infrastructure,clean technologies,low-emission technologies,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,SDG 9,9.5,innovative infrastructure,innovative industry,indsutrial innovation,industry technology,research and development,R&D,innovative technology,green technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,SDG 9,9.a,international support to infrastructure,development assistance to infrastructure,infrastructure in developing countries,infrastructure development in developing count...,developing countries infrastructure,oda for rural infrastructure development,official development assistance for rural infr...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,SDG 9,9.b,research in developing countries,innovation in developing countries,technology development in developing countries,technologies in developing countries,industry in developing countries,industrial diversification in developing count...,research and development developing countries,developing countries research and development,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Expand keywords

input = 'keywords/keywords_copia.xlsx'
keywords = pd.ExcelFile(input)
filename = pathlib.Path(input).stem
timestamp =  dt.datetime.now().isoformat(timespec='seconds').replace(':','').replace('T','_T')
filename += f'_{timestamp}'

output = f'{filename}.xlsx'

#sheets = a list of sheets where keywords are
#keys_df = [keywords.parse(sheet) for sheet in sheets]
keys_dfs = [keywords.parse('Target_keys'), keywords.parse('Goal_keys'), keywords.parse('MOI')]

for index, df in enumerate(keys_dfs):
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';;', ';', keywords))
    ex_df=df['Keys'].str.split(';', expand=True)
    lab_df=df.drop(columns='Keys')
    keys_dfs[index]=lab_df.join(ex_df)
    #df['Keys']=df['Keys'].apply(lambda keywords: re.sub(r'[^a-zA-Z0-9; -.]+', ';', keywords))

ex_df.head()

keys_dfs.append(keywords.parse('developing_countries'))

target_df, goal_df, devco_df, devco_names = keys_dfs

In [ ]:
target_df.head()

In [ ]:
ex_target_df=target_df['Keys'].str.split(';', expand=True)
lab_target_df=target_df.drop(columns='Keys')
lab_target_df.head()

In [ ]:
with pd.ExcelWriter(output, engine='xlsxwriter') as writer: # 'keywords_copy_for_transaltion.xlsx'
    target_df.to_excel(writer, sheet_name='Target_keys')
    goal_df.to_excel(writer, sheet_name='Goal_keys')
    devco_df.to_excel(writer, sheet_name='MOI')
    devco_names.to_excel(writer, sheet_name='developing_countries')

In [ ]:
#Load expand keywords

input = 'keywords/keywords_expanded.xlsx'

keywords = pd.ExcelFile(input)

sheets = keywords.sheet_names
print(f"Reading keywords dataset in {input}.\nSheets: {', '.join(sheets)}")

keywords = {sheet : keywords.parse(sheet, index_col=0) for sheet in sheets}
keywords_sheets = ['Target_keys', 'Goal_keys', 'MOI']

#remove all from stop_words to keep in keywords
stop_words = set(stopwords.words('english'))-set(['no','not','nor'])
stop_words.remove('all')

for sheet in keywords_sheets:
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: [plmp.preprocess_text(keyword, stop_words) for keyword in keywords.split(';')])
    keywords[sheet].iloc[:,2:]=keywords[sheet].iloc[:,2:].applymap(lambda keyword: plmp.preprocess_text(str(keyword), stop_words))

In [ ]:
keywords['Target_keys'].head()

In [ ]:
#Load text

input = pathlib.Path('input/52019DC0640_stemmed')

files = sorted(input.glob('**/*.*'))

print(files)

text_ls = {}

for _file in files:
    with open(_file, 'r') as _f:
        text_ls[_file.stem]=_f.read()

In [ ]:
def count_keyword(text, keyword):

count = text.count(keyword)


In [ ]:
keywords['Target_keys']

In [ ]:
text_df = []

texts = list(text_ls.values())

keyword_matrix = keywords['Target_keys'].iloc[:,2:]#.to_numpy()
count_matrix = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: texts[0].count(keyword))#.to_numpy()
labels = keywords['Target_keys'].iloc[:,0]

counted_keywords = count_matrix.mask(count_matrix > 0, keyword_matrix)
counted_keywords = counted_keywords.apply(lambda keyword : ', '.join(keyword.astype(str)))
count_matrix = count_matrix.replace(0, '')
counted_keywords.to_excel('key_col.xlsx')
count_matrix.to_excel('mask.xlsx')


# for name, text in text_ls.items():
#     count_df = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: text.count(keyword))
#     #count_matrix = count_df#.to_numpy()
#     count_df = count_df.join(labels, how='inner')
#     count_df['Sum_counts'] = count_df.sum(axis=0)
#     keywords_ = keyword_matrix[count_matrix>0]
#     #count_df = count_df[count_df['Sum_counts'] > 0]
#     # #df.replace(0, None, inplace=True)
#     #df.insert(loc=1, column='Label', value=keywords['Target_keys'].iloc[:,1])
#     # df.insert(loc=2, coulmn='Count_keys', value=df.apply(lambda x: sum(x),axis=1))
#     # df.insert(loc=3, column='Diff_keys', value=df.apply(lambda x:count(x),axis=1))
#     count_df.to_excel(f'test_{name}.xlsx')
#     np.savetxt(f'test_{name}.csv', count_matrix, delimiter=',', fmt='%i')
#     np.savetxt(f'test_{name}_keys.csv', keywords_, delimiter=',',fmt='%s')
#     # np.savetxt('test.csv', count_matrix, delimiter=',')
#     text_df.append(count_df)

In [ ]:
keyword_matrix.head()

In [ ]:
count_matrix=count_matrix.mask(count_matrix > 1, -1)
count_matrix.to_excel('mask.xlsx')

In [ ]:
input = 'output/results_green_deal_test.xlsx'

result_df = pd.ExcelFile(input)

print(result_df.sheet_names)

targets, goals, total = result_df.parse('target_count'), result_df.parse('goal_overview'), result_df.parse('total_count_(goals_+_targets)')

# print(result_df.sheet_names)

In [ ]:
targets.rename(columns={'Count':'Target_Count', 'Num_keys':'Tgt_nr.keys'}, inplace=True) 
targets.groupby(['Policy','Goal','Target']).sum()
dat_target_join_keys = raw_df.groupby(['Policy', 'Target'])["Keyword"].apply(lambda x : " - ".join(x.astype(str))).reset_index()

In [ ]:
targets.head

In [ ]:
goals.rename(columns = {'Count':'Goal_Count', 'Goal_nr.keys'}, inplace = True) 
goals.groupby(['Policy','Goal']).sum()